<img src="../figs/holberton_logo.png" alt="logo" width="500"/>

# Traffic Sign Detection 

## Executive Summary

The goal of traffic sign detection is to enable vehicles, particularly autonomous ones, to interpret and understand traffic signs on roads for safe and efficient navigation. Using computer vision techniques, traffic sign detection systems identify and classify various traffic signs such as speed limits, stop signs, and pedestrian crossings from images or video feeds captured by onboard cameras. By accurately recognizing and interpreting these signs, vehicles can make informed decisions, adhere to traffic regulations, and enhance road safety for both passengers and pedestrians.

## German Traffic Sign Dataset

To facilitate research in this area, the [German Traffic Sign Recognition Benchmark (GTSRB)](https://www.kaggle.com/datasets/harbhajansingh21/german-traffic-sign-dataset/data) dataset has been created. This dataset consists of images of traffic signs captured from German roads, with each image labeled according to its corresponding class. The dataset focuses on single-image, multi-class classification challenges and does not include any temporal information from the original video footage.

Here are some details about the GTSRB dataset:

1. Each image in the dataset has a resolution of *32 x 32 pixels and is represented in RGB format* with three color channels. The pixel values are stored as unsigned 8-bit integers, providing a total of 256 possible values for each pixel.

2. The dataset comprises a total of *43 distinct classes or labels*, based on the design or meaning of the traffic signs.

3. The *training set consists of 34,799 images*, each associated with its corresponding label.

4. The *validation set contains 4,410 images*, also accompanied by their respective labels.

5. Lastly, the *test set comprises 12,630 images*, each labeled with its corresponding class.

In CSV file,there are two columns

- `ClassID` (represents the class ID of traffic signs) 
- `SignName` (represents the name of traffic sign).

### 0. Load the Data

We begin by importing the necessary libraries: `pickle` for loading the data from pickle files and `numpy` for handling arrays and data manipulation.

Next, we define the path to the directory where the pickle files are stored. The variable `pickle_path` holds the path to the directory containing the files.

We define a function `load_pickle` to load the data from a pickle file. This function takes the file path as input and returns the features and labels contained in the file.

Using this function, we load the training data from the file `train.p`, the test data from `test.p`, and the validation data from `valid.p`. We call the function load_pickle for each file, passing the file path as an argument.



In [1]:
import pickle
import numpy as np

# Assuming your files are placed inside a folder called trafficsigns
# at the same directory as this notebooke
pickle_path = "trafficsigns/"

"""
If you are using Google colab, you need to *mount* the drive first. You can use

drive.mount('/content/drive', force_remount=True)
pickle_path = '/content/drive/MyDrive/trafficsigns/'  

assuming that all files are uploaded on your Google drive, inside a folder called trafficsigns
"""

# Function to load pickle file
def load_pickle(file_path):
    with open(file_path, mode='rb') as file:
        dataset = pickle.load(file)
    return dataset['features'], dataset['labels']

# Load train data
train_data, train_labels = load_pickle(pickle_path + "train.p")
print("Train data shape:", train_data.shape)
print("Train labels shape:", train_labels.shape)

# Load test data
test_data, test_labels = load_pickle(pickle_path + "test.p")
print("Test data shape:", test_data.shape)
print("Test labels shape:", test_labels.shape)

# Load validation data
valid_data, valid_labels = load_pickle(pickle_path + "valid.p")
print("Validation data shape:", valid_data.shape)
print("Validation labels shape:", valid_labels.shape)


Train data shape: (34799, 32, 32, 3)
Train labels shape: (34799,)
Test data shape: (12630, 32, 32, 3)
Test labels shape: (12630,)
Validation data shape: (4410, 32, 32, 3)
Validation labels shape: (4410,)


As we can denote from printing the shape of the data and labels for each dataset (train, test, and validation), to verify that the data has been loaded correctly. This step ensures that the data is in the expected format and size before proceeding with further analysis or modeling

### 1. Visualize Data


We define a function `visualize_random_data` that takes the data and labels as input and visualizes random samples from the dataset. It creates a grid of subplots with the specified number of rows and columns (`num_rows` and `num_cols`). For each subplot, it randomly selects an index from the dataset, displays the corresponding image, and sets the title as the label. Finally, it shows the plot containing the visualizations.


#### Understanding class categories

To better understand the class categories, we can load the `signnames.csv` file, which contains the categories of traffic signs along with their corresponding labels.

The major types of traffic signs include speed limits, regulatory signs such as "No passing" and "Stop", priority signs like "Yield" and "Priority road", prohibition signs such as "No entry" and "No vehicles", warning signs like "General caution" and "Road work", and informational signs such as "Traffic signals" and "Pedestrians". Additionally, there are signs indicating road conditions like "Slippery road" and "Bumpy road", as well as signs providing guidance on navigation like "Turn right ahead" and "Go straight or left".

#### Displaying Category Distribution

The analysis of the training data reveals a notable imbalance in the distribution of images across different traffic sign categories. While some categories have a substantial number of images, others are significantly underrepresented. This class imbalance can potentially hinder the performance of the learning algorithm during training. Classes with fewer samples may not receive sufficient attention during model optimization, leading to biased predictions and reduced overall accuracy. Therefore, addressing class imbalance through techniques like data augmentation, resampling, or weighted loss functions could be crucial for improving the robustness and generalization ability of the learning algorithm.

### 2. Build and Compile Model 

To build a good CNN architecture for the traffic sign classification task, we need to design a model that can effectively capture the spatial hierarchies and intricate features present in the images. Here's a proposed architecture


- The model begins with a convolutional layer with 32 filters of size (3, 3) and a ReLU activation function. This layer extracts basic features from the input images.
- Max pooling layers with a pool size of (2, 2) follow each convolutional layer to downsample the feature maps and reduce computational complexity.
- Dropout layers are included after each max pooling layer to prevent overfitting by randomly dropping a fraction of the neurons during training.
- The depth of feature maps is progressively increased with subsequent convolutional layers (64 and 128 filters).
- After the final convolutional layer, the output is flattened to a 1D array to be fed into the fully connected layers.
- Two dense fully connected layers with 512 neurons each and ReLU activation functions are added to learn higher-level features.
- A dropout layer with a dropout rate of 0.5 is inserted before the output layer to further prevent overfitting.
- The output layer consists of 43 neurons (equal to the number of traffic sign classes) with a softmax activation function, producing probabilities for each class.

This architecture strikes a balance between model complexity and generalization ability, making it suitable for the traffic sign classification task. The model is trained using the Adam optimizer with a sparse categorical cross-entropy loss function and evaluated based on accuracy metrics


### 3. Train Model

In this part, we train the CNN architecture using the training data while validating the model's performance on the validation data. 


The training process consists of multiple epochs (20 in this case), where the model iteratively learns to classify traffic signs by adjusting its internal parameters based on the provided images and labels. 


We use a batch size of 128, meaning the model is updated after processing each batch of 128 training samples. 


Additionally, early stopping is implemented with a patience of 3 epochs, meaning training will stop if the validation loss does not improve for 3 consecutive epochs. The `restore_best_weights=True` argument ensures that the model's weights are restored to the best configuration observed during training when early stopping is triggered. 


This helps prevent overfitting and ensures the model's generalization ability. Throughout training, the model's performance metrics (loss and accuracy) are monitored on both the training and validation datasets, providing insights into its learning progress and generalization ability



#### Visualize Training Performance

We will generate two subplots side by side. The subplot on the left will display the training and validation loss over epochs, while the subplot on the right will display the training and validation accuracy over epochs. Adjust the plot settings as needed for better visualization.

The visualizations of training and validation performance metrics reveal a similarity, with both accuracy and loss curves tracking closely throughout the training process. This **consistency** indicates that the model has been **effectively trained** and is demonstrating **strong generalization** ability. The close alignment between the training and validation curves suggests that the **model is not overfitting** to the training data and is capable of accurately classifying traffic signs on unseen validation data. These findings indicate a well-trained model with **robust performance** on both training and validation datasets.

### 4. Predict 

The goal of prediction is to use the trained model to classify traffic signs in unseen data accurately. This involves passing the test data through the trained model and comparing the predicted labels with the actual labels to evaluate the model's performance.

### 4.1 Classification Reports and Accuracy Scores



### Summary

Throughout this development process, we **constructed and trained a Convolutional Neural Network (CNN) architecture to classify traffic signs effectively**. 


By leveraging the **German Traffic Sign Dataset**, we loaded and preprocessed the training, validation, and test data, ensuring that the model could learn from diverse samples. 


Employing a **CNN architecture comprising convolutional, max pooling, dropout, and dense layers**, we established a robust framework capable of extracting intricate features from traffic sign images. 

Training the model on the training data and validating it on a separate validation dataset, we ensured its generalization ability and mitigated overfitting through early stopping. Visualizing the training and validation metrics provided insights into the model's learning progress and confirmed its well-trained status. 

Finally, evaluating the model on unseen test data yielded a commendable accuracy of 96.11%, affirming the effectiveness of the developed CNN architecture in accurately classifying traffic signs.